<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Факторизация матрицы рейтингов и </b> <span style="font-weight:bold; color:green">Spark MLlib</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_bmstu@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Подключение библиотек и создание Spark контекста</a></li>
        <li><a href="#2">Загрузка исходных данных</a></li>
        <li><a href="#3">Анализ исходных данных</a></li>
        <li><a href="#4">Факторизация матрицы рейтингов</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#4a">Формирование обучающего и тестового подмножеств</a></li>
                <li><a href="#4b">Вычисление базовой отметки</a></li>
                <li><a href="#4c">Факторизация матрицы с использованием ALS</a></li>
            </ol>
        </li>
        <li><a href="#5">Завершение работы</a></li>
    </ol>
</div>

<p>Подключение стилей оформления</p>

In [ ]:
%%html
<link href="css/style.css" rel="stylesheet" type="text/css">

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Подключение библиотек и создание Spark контекста</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>[ОПЦИОНАЛЬНО] <b>Настройка среды</b></p>

In [ ]:
import os
import sys

os.environ["SPARK_HOME"]="/opt/cloudera/parcels/SPARK2/lib/spark2"
os.environ["PYSPARK_PYTHON"]="/opt/rh/rh-python36/root/usr/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]="/opt/rh/rh-python36/root/usr/bin/python"

spark_home = os.environ.get("SPARK_HOME")
sys.path.insert(0, os.path.join(spark_home, "python"))
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.10.7-src.zip"))

<p>Запуск Spark Session</p>

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
conf = pyspark.SparkConf() \
        .setAppName("moviewRecomApp") \
        .setMaster("yarn") \
        .set("spark.submit.deployMode", "client")

In [ ]:
spark = SparkSession \
    .builder \
    .appName("moviewRecomApp") \
    .config(conf=conf) \
    .getOrCreate()

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Загрузка исходных данных</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>[ЕСЛИ НЕОБХОДИМО]</p>

In [ ]:
!hdfs dfs -mkdir -p data/spark_mllib

In [ ]:
!hdfs dfs -copyFromLocal data/movie-recommendation-data/ml-latest-small/movies.csv data/spark_mllib/movies.csv
!hdfs dfs -copyFromLocal data/movie-recommendation-data/ml-latest-small/ratings.csv data/spark_mllib/ratings.csv

In [ ]:
!hdfs dfs -ls data/spark_mllib

<p>Путь к данными</p>

In [ ]:
# Доступ к данным из HDFS для небольшего набора данных
movie_data_path = "data/spark_mllib/movies.csv"
ratings_data_path = "data/spark_mllib/ratings.csv"

# Databricks
#movie_data_path = "/FileStore/tables/movies.csv"
#ratings_data_path = "/FileStore/tables/ratings.csv"

# Доступ к данным из HDFS для полного набора данных
# movie_data_path = "hdfs:///data/recom/large/movies.csv"
# ratings_data_path = "hdfs:///data/recom/large/ratings.csv"

<div class="msg-block msg-info">
  <div class="msg-text-info">
      <p>Данные необходимо предварительно загрузить. Ссылки на данные:<br>
      1. Небольшой набор: http://files.grouplens.org/datasets/movielens/ml-latest.zip <br>
      2. Полный набор: http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
      </p>
  </div>
</div>

<div class="msg-block msg-info">
  <div class="msg-text-info">
      <p>1. Выполните последующие шаги сначала для небольшого набора данных и только потом для полного.<br>
         2. Для загрузки данных из локальной файловой системы используйте префикс <span class="code-font">file:///</span> 
      </p>
  </div>
</div>

<p>Фильмы</p>

<div class="msg-block msg-warning">
  <p class="msg-text-warn">При использовании <span class="code-font">file:///</span> метод <span class="code-font">sqlContext.read.load()</span> может выдать ошибку</p>
</div>

In [ ]:
df_movies = spark.read.load(movie_data_path, 
                            format="csv", 
                            header="true", 
                            inferSchema="true", 
                            sep=",")
df_movies.show(5)

<p>Рейтинги</p>

In [ ]:
df_ratings = spark.read.load(ratings_data_path,          
                             format="csv", 
                             header="true", 
                             inferSchema="true", 
                             sep=",")
df_ratings.show(5)

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Анализ исходных данных</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>Фильмы с наибольшим количеством просмотров</p>

In [ ]:
group = df_ratings.groupBy("movieId")

In [ ]:
df_movie_count = group.agg({"movieId": "count", "rating":"mean"})
df_movie_count.show(10)

In [ ]:
df_movie_count_sorted = df_movie_count.sort("count(movieId)", ascending=False)
df_movie_count_sorted.show(10)

In [ ]:
df_movie_count_title = df_movie_count_sorted.join(other=df_movies, on="movieId", how="inner")
df_movie_count_title.show(10)

<p>Фильмы с наименьшим количеством просмотров</p>

In [ ]:
df_movie_count_title.sort("count(movieId)", ascending=True).show(10)

<p>Фильмы с наилучшими средними рейтингами</p>

In [ ]:
df_movie_count_title.sort("avg(rating)", ascending=False).show(10)

In [ ]:
df_movie_count_title.sort("avg(rating)", ascending=False) \
                    .filter(df_movie_count_title["count(movieId)"] > 100) \
                    .show(10)

<p>Фильмы с наихудшими средними рейтингами</p>

In [ ]:
df_movie_count_title.sort("avg(rating)", ascending=True).show(10)

In [ ]:
df_movie_count_title.sort("avg(rating)", ascending=True) \
                    .filter(df_movie_count_title["count(movieId)"] > 50) \
                    .show(10)

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4. Факторизация матрицы рейтингов</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name="4a"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Формирование обучающего и тестового подмножеств
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#4">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#4b">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
df_train, df_test = df_ratings.randomSplit([0.9, 0.1], seed=12)
df_train.persist(); df_test.persist()

In [ ]:
df_train.count(), df_test.count()

In [ ]:
df_train.rdd.getNumPartitions()

<a name="4b"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Вычисление базовой отметки
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#4a">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#4c">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from pyspark.sql.functions import mean, lit, sum, count, col

In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

<p>Вычисление среднего рейтинга</p>

<p><i>Способ 1</i></p>

In [ ]:
mean_movie_rating = df_train.select(mean("rating").alias("avr")).collect()[0]["avr"]
mean_movie_rating

<p><i>Способ 2</i></p>

In [ ]:
df_train_descr = df_train.select("rating").describe()
df_train_descr.show()

In [ ]:
df_train_descr.collect()

In [ ]:
df_train_descr.toPandas()

<p>Тестирование</p>

<p><i>Формирование столбца с предсказанием</i></p>

In [ ]:
df_test_pred_bl = df_test.withColumn("prediction", lit(mean_movie_rating)) # broadcast mean_movie_rating
df_test_pred_bl.show(5)

<p><i>Вычисление RMSE</i></p>

In [ ]:
eval_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [ ]:
eval_rmse.evaluate(df_test_pred_bl)

<a name="4c"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            c. Факторизация матрицы с использованием ALS
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#4b">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#5">Далее</a>
            </div>
        </div>
    </div>
</div>

<p><b>Обучение</b></p>

<p>Применение ALS для предсказания рейтингов</p>

In [ ]:
als_inst = ALS(rank=10, maxIter=10, seed=123, regParam=0.1, 
               numUserBlocks=10, numItemBlocks=10, 
               userCol="userId", itemCol="movieId", ratingCol="rating")

m_als = als_inst.fit(df_train)
m_als

<p>Предсказания при обучающем подмножестве</p>

In [ ]:
train_pred = m_als.transform(df_train)
train_pred.show(5)

<p>RMSE</p>

In [ ]:
eval_rmse.evaluate(train_pred)

<p><b>Проверка на тестовом подмножестве</b></p>

<p>Предсказания при тестовом подмножестве</p>

In [ ]:
test_pred = m_als.transform(df_test)
test_pred.show(5)

<p>Проблема</p>

In [ ]:
eval_rmse.evaluate(test_pred)

<p>Причина</p>

In [ ]:
test_pred.sort("prediction", ascending=False).show(10)

<p>Вариант решения</p>

In [ ]:
test_pred_with_mean = test_pred.fillna(mean_movie_rating)

In [ ]:
test_pred_with_mean.sort("prediction", ascending=False).show(10)

<p>RMSE</p>

In [ ]:
eval_rmse.evaluate(test_pred_with_mean)

<p>Рекомендации пользователю</p>

In [ ]:
user_id = 463
num_recom = 10 # количество рекомендаций

<p>Рейтинги пользователя</p>

In [ ]:
df_ratings.where(col("userId")==user_id) \
            .join(other=df_movies, on="movieId", how="inner") \
            .sort("rating", ascending=False) \
            .show()

<p>Рекомендации</p>

In [ ]:
recom_movies_for_users_df = m_als.recommendForAllUsers(num_recom)
recom_movies_for_users_df.show(5)

In [ ]:
recom_movies_for_users_df.where(col("userId")==user_id) \
                            .select("recommendations.movieId", "recommendations.rating") \
                            .show()

In [ ]:
movies_pandas = recom_movies_for_users_df.where(col("userId")==user_id).select("recommendations.movieId").toPandas()
movies_pandas["movieId"]

In [ ]:
movies_list = movies_pandas["movieId"][0]
movies_list

In [ ]:
sc = spark.sparkContext
movies_list_broadcast = sc.broadcast(movies_list)

In [ ]:
df_movies.filter(col("movieId").isin(movies_list_broadcast.value)).show()

<a name="5"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">5. Завершение работы</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
spark.stop()